#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [ ]:
import sys
import os

# Ensure src folder is discoverable
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")
sys.path.append(src_path)

In [ ]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [10]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1*365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [ ]:
# import os, sys
# # Detect where we’re running from
# notebook_dir = os.getcwd()

# # If running from GitHub Actions (root dir), go into src directly
# if os.path.exists("src/get_stocks_data.py"):
#     sys.path.append("src")
# # If running from notebooks folder locally
# elif os.path.exists("../src/get_stocks_data.py"):
#     sys.path.append("../src")

# import get_stocks_data as gs

In [ ]:
# import sys
# sys.path.append("../src")

import get_stocks_data as gs

db_path = "../data/stocks.db"

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["RELIANCE.NS", "HDFCBANK.NS", "TCS.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-10-07 13:06:55,300 - INFO - 📈 Fetching data for 2178 tickers...
2025-10-07 13:06:55,301 - INFO - Fetching data from 2024-09-07 to 2025-10-07


2178


[*********************100%***********************]  2178 of 2178 completed
2025-10-07 13:08:13,280 - ERROR - 
1 Failed download:
2025-10-07 13:08:13,281 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-10-07 13:08:16,553 - INFO - ✅ Download completed in 81.25 seconds
2025-10-07 13:08:20,539 - INFO - 📊 Final dataset: 559,414 rows from 2024-09-09 to 2025-10-07


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2024-09-09,20MICRONS.NS,NaN,313.534027,319.457258,305.818870,314.031779,193859.0,313.534027,313.534027,313.534027,313.534027,NaN,NaN,20MICRONS
1,2024-09-10,20MICRONS.NS,NaN,311.095032,318.810189,310.099528,313.583792,131088.0,313.301742,313.438380,313.485730,313.509758,NaN,NaN,20MICRONS
2,2024-09-11,20MICRONS.NS,NaN,306.067749,313.832669,303.977184,310.099528,135469.0,312.612790,313.149336,313.338839,313.435709,NaN,NaN,20MICRONS
3,2024-09-12,20MICRONS.NS,NaN,302.981659,311.543000,301.637722,308.805364,162769.0,311.695540,312.750603,313.133747,313.331688,NaN,NaN,20MICRONS
4,2024-09-13,20MICRONS.NS,NaN,307.212555,313.384662,303.529178,303.529178,204174.0,311.268589,312.533425,313.016495,313.270801,NaN,NaN,20MICRONS


#### Call Accumulation signal

In [ ]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


acs.save_accumulation_to_db(accum_df, db_path)

# View
accum_df.head()

,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,AEGISLOG,2025-10-06,475817.6134,75.0,1.93,60.22,False,772.426114,752.576245,762.863611,831.000000,2.403842e+06,1.331013e+06
1,CYBERTECH,2025-10-07,388919.1864,85.0,3.14,68.18,False,178.631677,161.084093,189.524659,203.220001,4.052305e+06,9.956904e+05
2,ATLANTAA,2025-10-06,286662.4505,90.0,1.99,84.79,True,43.528996,39.259315,39.368980,66.779999,1.866049e+06,7.913913e+05
3,AFIL,2025-10-06,252613.4430,95.0,2.16,65.64,True,8.273954,7.775811,8.004169,8.940000,1.058086e+07,4.235569e+06
4,ATL,2025-10-06,186450.2602,70.0,2.25,76.16,False,33.954498,31.571360,31.835408,37.189999,1.337318e+06,7.228355e+05


In [ ]:
# accum_df.to_csv(f"D:/Work/Projects/Stock_Market/Stock_Market/Turn_Around_Stocks/signal_outputs/Turn_Around_stocks1_{df.Date.max().strftime('%Y-%m-%d')}.csv", index = False)

#### EMA Signal indicator